<b><div style="text-align:right">Алексей Бебчик&nbsp;&nbsp;</div><b>
### Курс 6. Желтое такси. Неделя 6. Дополнительные признаки  
(задание, оцениваемое сокурсниками)





<a href="https://www.coursera.org/learn/data-analysis-project/peer/gtNEG/dopolnitiel-nyie-priznaki">Задание</a>

На этой неделе вам предстоит попробовать добавить в вашу регрессионную модель дополнительные признаки.

Во-первых, для прогнозирования можно использовать информацию, содержащуюся в сырых данных:
- средняя длительность поездок;  
- среднее количество пассажиров;  
- среднее расстояние по счётчику;  
- доли географических зон, в которые совершаются поездки;  
- доли поездок, совершаемых по тарифам каждого из типов;  
- доли способов оплаты поездок;  
- средняя стоимость поездок;  
- доли провайдеров данных.

Все эти признаки можно использовать только с задержкой, то есть, при прогнозировании $\hat{y}_{T+i|T}$ эти признаки должны быть рассчитаны по данным не позднее момента времени $T$. Каждый из этих признаков можно использовать по-разному: как сырые значения за последние несколько часов, так и средние за последний день, неделю, месяц и т.д.  

Во-вторых, чтобы улучшить качество прогнозов в аномальные периоды, вы можете найти информацию о потенциально влияющих на количество поездок событиях, таких, как государственные праздники. Проанализируйте, как именно поведение пассажиров меняется во время этих событий, и создайте признаки, отражающие эти изменения. Как показывает наш опыт, правильный учёт праздничных дней часто позволяет существенно уменьшить среднюю ошибку прогноза.  

В-третьих, можно использовать признаки, связанные с географией. Например, скорее всего, суммарное количество поездок, совершаемых из географической зоны, пропорционально площади этой зоны. Для зон, прилегающих к аэропорту, может быть характерен специфический паттерн дневной сезонности, связанный с тем, что спрос на такси будет повышаться в те часы, когда общественный транспорт перестаёт работать. В деловом центре максимальное количество поездок будет приходиться на начало и окончание рабочего дня, на Бродвее — на время начала и окончания спектаклей. Все эти идеи не обязательно верны, мы приводим их здесь только для того, чтобы продемонстрировать принцип рассуждений. Ещё один пример географического признака: можно попробовать добавить идентификатор боро, который можно найти в файле https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv. Кроме того, нам кажется перспективным использование в качестве фактора количества поездок, совершённых за прошлый час/день и т. д. из соседних географических зон, или количества поездок, совершённых за прошлый час/день в текущую географическую зону.    

Много примеров других признаков, которые можно использовать при регрессионном прогнозировании, можно найти в лекции Вадима Стрижова.  

Чтобы сдать задание, выполните следующую последовательность действий.    

Загрузите обучающие выборки прошлой недели, перечислите используемые в моделях признаки и посчитайте $Q_{may}$ — качество прогнозов моделей, настроенных на данных до апреля 2016, в мае 2016.  

Попробуйте добавить признаки. Используйте идеи, которые мы предложили, или какие-то свои. Обучайте обновлённые модели на данных до апреля 2016 включительно и считайте качество новых прогнозов на мае. Удаётся ли вам улучшить качество? Не нужно ли увеличить сложность регрессионной модели? Если добавляемый признак не улучшает качество, всё равно оставьте доказательства этому в ноутбуке, чтобы ваши коллеги это видели при проверке.   

Когда вы примете решение остановиться и перестать добавлять признаки, постройте для каждой географической зоны и каждого конца истории от 2016.04.30 23:00 до 2016.05.31 17:00 прогнозы на 6 часов вперёд; посчитайте в ноутбуке ошибку прогноза по следующему функционалу:  
$Q_{may} =\frac1{R* 739* 6} \sum\limits_{r=1}^{R} \sum_{T=2016.04.30 23:00}^{2016.05.31 17:00} \sum_{i=1}^6 \left| \hat{y}^r_{T|T+i} - y^r_{T+i} \right|$.  

Убедитесь, что среднее качество прогнозов увеличилось.
Переобучите итоговые модели на данных до мая 2016 включительно, постройте прогнозы на июнь для каждого конца истории от 2016.05.31 23:00 до 2016.06.30 17:00 и запишите все результаты в один файл в уже знакомом вам формате: $geoID, histEndDay, histEndHour, step, y$  

Загрузите полученный файл на kaggle: https://inclass.kaggle.com/c/yellowtaxi. Добавьте в ноутбук ссылку на сабмишн.
Загрузите ноутбук в форму.  

__Review criteria__  
В качестве ответа в этом задании вам нужно загрузить ноутбук; убедитесь, что ход анализа, который вы провели, описан достаточно подробно для того, чтобы ваши сокурсники поняли, что вы делали и почему. 

In [14]:
#%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import binned_statistic_2d
import os                                     #для проверки существования файлов
import itertools
import statsmodels.api as sm
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn import linear_model, ensemble, metrics
import datetime as dt
# import seaborn as sns 
import timeit as ti
from tqdm import tqdm_notebook, tqdm
from time import sleep

from datetime import date
import holidays

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [15]:
data_path  = '..\\YTData\\'  #путь к папке с данными "K6w6_290718"
data6_path = 'YTw6_data\\'   #путь к папке с данными 

In [16]:
def print_time(label='now'):
    print label, dt.datetime.now().strftime('%H:%M')

In [18]:
#Загружаем подготовленные данные из файла
data_ext = pd.read_csv(data6_path + 'YTw6_data_ext.csv', parse_dates=['dt_hour'], dayfirst=True) 
list_region = list(data_ext.region.unique())

In [12]:
data0 = pd.read_csv(data6_path + 'YTw6_data0.csv', parse_dates=['dt_hour'], dayfirst=True) #Загружаем подготовленные данные из файла
list_region = list(data0.region.unique())
# Составьте из данных о поездках прямоугольную таблицу так, чтобы по строкам было время, а по столбцам идентификатор ячейки
data = pd.pivot_table(data0, values='trips', index =['dt_hour'], columns=['region'])

In [13]:
data.head()

region,1075,1076,1077,1125,1126,1127,1128,1129,1130,1131,...,1630,1684,1733,1734,1783,2068,2069,2118,2119,2168
dt_hour,,,,,,,,,,,,,,,,,,,,,
2016-01-01 00:00:00,80,144,50,77,319,402,531,617,846,267,...,12,0,2,44,5,41,4,70,7,66
2016-01-01 01:00:00,91,211,49,134,404,420,370,453,594,224,...,29,0,5,2,2,4,0,47,1,29
2016-01-01 02:00:00,90,146,23,110,393,425,313,366,377,138,...,47,0,3,0,4,0,0,69,1,14
2016-01-01 03:00:00,32,87,16,62,252,399,324,309,327,166,...,46,0,2,4,5,1,0,21,0,9
2016-01-01 04:00:00,24,43,10,53,145,254,264,333,318,145,...,43,0,0,1,1,0,0,26,1,6


In [24]:
data_values = data.values

__ЧАСТЬ 1. Признаки__

Для каждой из шести задач прогнозирования $ŷ T+i∣T,i=1,…,6$ $y^T+i∣T,i=1,…,6$ сформируйте выборки. Откликом будет  $yT+i$ при всевозможных значениях $T$ , а признаки можно использовать следующие:  
• идентификатор географической зоны — категориальный;  
• год, месяц, день месяца, день недели, час — эти признаки можно пробовать брать и категориальными, и непрерывными, можно даже и так, и так;  
• синусы, косинусы и тренды, которые вы использовали внутри регрессионной компоненты ARIMA;  
• сами значения прогнозов ARIMA $\hat{y}_{T+i|T}^{ARIMA}$;  
• количество поездок из рассматриваемого района в моменты времени $y_{T},y_{T-1}, \dots, y_{T-K}$ (параметр $K$ можно подбирать; попробуйте начать, например, с 6);  
• количество поездок из рассматриваемого района в моменты времени $y_{T-24}, y_{T-48}, \dots, y_{T-24*K_d}$ (параметр $K_d$ можно подбирать; попробуйте начать, например, с 2);  
• суммарное количество поездок из рассматриваемого района за предшествующие полдня, сутки, неделю, месяц.  


In [3]:
train_endtime = '2016.04.30 17:00'
tune_begtime  = '2016.04.30 23:00'
tune_endtime  = '2016.05.31 17:00'
test_begtime  = '2016.05.31 23:00'
test_endtime  = '2016.06.30 17:00'

In [4]:
pd.set_option('display.max_columns', 250)
pd.set_option('display.max_rows', 250)

In [226]:
actual_regs = pd.read_csv('k6w6_reg.csv')

Схема формирования признаков сформирована по мотивам работы Ушакова Вадима Константиновича. Предложенный коллегой подход прост и эффективнен, так что представляется полезным использовать его в жизни.

In [27]:
K = 23 # количество поездок за К часов
K_d = 7 # количество поездок за К*24 часа
k_week = 4 # количество Фурье компонент для недели
k_pol = 3 # степень полинома
Y1 = []
Y2 = []
Y3 = []
Y4 = []
Y5 = []
Y6 = []
times = []
regions = []
years = []
months = []
weeks = []
hours = []
hour_trips = []
day_trips = []
total_half_day = []
total_day = []
total_week = []
total_month = []
sin_week = []
cos_week = []
polinom = []
weather = []

#для каждого района
for region in tqdm_notebook(range(102)):
    #для каждой строки данных го ЧЕГО: От (ЧЕГО до КонцаМинус6Часов)
    for j in range(1, len(data.index)-6): #range(len(data.index) - 8760*2, len(data.index)-6)
        T = data.index[j] #ДатаВремя
        times.append(T)  #Индекс-ДатаВремя
        series = data_values[:j, region]
        t = len(series)
        # Фурье компоненты недели
        sin_week.append([np.sin(t * 2 * pi * (i+1) / 168) for i in range(k_week)])
        cos_week.append([np.cos(t * 2 * pi * (i+1) / 168) for i in range(k_week)])
        polinom.append([t**(i+1) for i in range(k_pol)])
        regions.append(list_region[region]) # идентификатор географической зоны
        years.append(T.year)      # год
        months.append(T.month)    # месяц
        weeks.append(T.weekday()) # день недели
        hours.append(T.hour)      # час

        hour_trips.append([series[-1-t1] if len(series)-1-t1>=0 else -1 for t1 in range(K)]) # количество поездок за К часов

        day_trips.append([series[-1-(t+1)*24] if len(series)-1-(t+1)*24>=0 else -1 for t in range(K_d)]) # количество поездок за К*24 часа
        
        total_half_day.append(series[-12:].sum()) # суммарное количество поездок за 12 часов
        total_day.append(series[-24:].sum()) # суммарное количество поездок за день
        total_week.append(series[-168:].sum()) # суммарное количество поездок за неделю
        total_month.append(series[-730:].sum()) # суммарное количество поездок за месяц
        
        # Для ускорения создаем сразу все 6 таргетов 
        #!!!признаки используются как samples[:,-6])!!!
        Y1.append(data_values[j+1, region])
        Y2.append(data_values[j+2, region])
        Y3.append(data_values[j+3, region])
        Y4.append(data_values[j+4, region])
        Y5.append(data_values[j+5, region])
        Y6.append(data_values[j+6, region])

In [28]:
columns = ['region', 'year','month','weekday','hour']
columns.extend(['hour_trips'+str(i+1) for i in range(K)])
columns.extend(['day_trips'+str(i+1) for i in range(K_d)])
columns.extend(['sin_week'+str(i+1) for i in range(k_week)])
columns.extend(['cos_week'+str(i+1) for i in range(k_week)])
columns.extend(['polinom'+str(i+1) for i in range(k_pol)])    
columns.extend(['total_half_day', 'total_day','total_week','total_month','y1','y2','y3','y4','y5','y6'])

In [29]:
cols_main_num = ['region', 'year','month','weekday','hour']
cols_hour_val = ['hour_trips'+str(i+1) for i in range(K)]
cols_day_val  = ['day_trips'+str(i+1) for i in range(K_d)]
cols_sincos   = ['sin_week'+str(i+1) for i in range(k_week)] + ['cos_week'+str(i+1) for i in range(k_week)]
cols_polinom  = ['polinom'+str(i+1) for i in range(k_pol)] 
cols_regsum   = ['total_half_day', 'total_day','total_week','total_month']
cols_y        = ['y1','y2','y3','y4','y5','y6']

In [30]:
all_cols = cols_main_num + cols_hour_val + cols_day_val + cols_sincos + cols_polinom + cols_regsum + cols_y

In [31]:
%%time
all_vals = np.column_stack((regions,years,months,weeks,hours,hour_trips,day_trips,
                           sin_week,cos_week, polinom, total_half_day,total_day,total_week,total_month,Y1,Y2,Y3,Y4,Y5,Y6))

Wall time: 3.3 s


In [32]:
%%time
data5_num = pd.DataFrame(all_vals, columns=all_cols,index=times) 

Wall time: 1.66 s


In [33]:
# Категориальные признаки 
data5_cat = pd.get_dummies(data5_num[cols_main_num], columns=cols_main_num)
cols_main_cat = list(data5_cat.columns) #считываем получившиеся столбцы

In [34]:
# data5 = pd.read_csv('k6w6_data5.csv', parse_dates=['dt_hour'], dayfirst=True) #Загружаем подготовленные данные из файла
# data5.index = data5load.dt_hour
# data5.index.name = None

In [35]:
data5 = pd.concat([data5_num, data5_cat], axis=1)
data5['region']  = data5.region.astype(int)
data5['dt_hour'] = data5.index

In [229]:
data5.to_csv(data6_path + 'YTw6_data5.csv', index=False)  #сохраняем признаки

In [36]:
data6 = pd.merge(data5, data_ext, on = ['dt_hour','region']).fillna(0)
data6.index = data6.dt_hour
data6.index.name = None

In [232]:
data6.to_csv('k6w6_data6.csv', index=False)  #сохраняем признаки

In [37]:
print data_ext.shape, data5.shape, data6.shape #(445536, 21) (444822, 197) (444822, 216)

(445536, 21) (444822, 197) (444822, 216)


In [5]:
data5 = pd.read_csv(data6_path + 'YTw6_data5.csv', parse_dates=['dt_hour'], dayfirst=True) #Загружаем подготовленные данные
data5.index = data5.dt_hour
data5.index.name = None

data6 = pd.read_csv(data6_path + 'YTw6_data6.csv', parse_dates=['dt_hour'], dayfirst=True) #Загружаем подготовленные данные
data6.index = data6.dt_hour
data6.index.name = None

__Добавим новые признаки, извлеченные из данных (файл k6w6_append_data.ipynb)__  
средняя длительность поездок;   
среднее количество пассажиров;   
среднее расстояние по счётчику;   
доли поездок, совершаемых по тарифам каждого из типов (категориальный);  
доли способов оплаты поездок (категориальный);                            
средняя стоимость поездок;   
доли провайдеров данных (категориальный).                                   

In [6]:
cols_y        = ['y1','y2','y3','y4','y5','y6']
cols_polinom  = ['polinom1', 'polinom2', 'polinom3']
cols_main_num = ['region', 'year', 'month', 'weekday', 'hour']
cols_main_cat = ['region_1075.0', 'region_1076.0', 'region_1077.0', 'region_1125.0', 'region_1126.0', 'region_1127.0', 'region_1128.0', 'region_1129.0', 'region_1130.0', 'region_1131.0', 'region_1132.0', 'region_1172.0', 'region_1173.0', 'region_1174.0', 'region_1175.0', 'region_1176.0', 'region_1177.0', 'region_1178.0', 'region_1179.0', 'region_1180.0', 'region_1181.0', 'region_1182.0', 'region_1183.0', 'region_1184.0', 'region_1221.0', 'region_1222.0', 'region_1223.0', 'region_1224.0', 'region_1225.0', 'region_1227.0', 'region_1228.0', 'region_1229.0', 'region_1230.0', 'region_1231.0', 'region_1232.0', 'region_1233.0', 'region_1234.0', 'region_1235.0', 'region_1272.0', 'region_1273.0', 'region_1274.0', 'region_1278.0', 'region_1279.0', 'region_1280.0', 'region_1281.0', 'region_1282.0', 'region_1283.0', 'region_1284.0', 'region_1285.0', 'region_1286.0', 'region_1287.0', 'region_1326.0', 'region_1327.0', 'region_1331.0', 'region_1332.0', 'region_1333.0', 'region_1334.0', 'region_1335.0', 'region_1336.0', 'region_1337.0', 'region_1338.0', 'region_1339.0', 'region_1376.0', 'region_1377.0', 'region_1378.0', 'region_1380.0', 'region_1382.0', 'region_1383.0', 'region_1384.0', 'region_1385.0', 'region_1386.0', 'region_1387.0', 'region_1388.0', 'region_1389.0', 'region_1390.0', 'region_1426.0', 'region_1431.0', 'region_1434.0', 'region_1435.0', 'region_1436.0', 'region_1437.0', 'region_1438.0', 'region_1439.0', 'region_1441.0', 'region_1442.0', 'region_1480.0', 'region_1482.0', 'region_1483.0', 'region_1530.0', 'region_1532.0', 'region_1533.0', 'region_1580.0', 'region_1630.0', 'region_1684.0', 'region_1733.0', 'region_1734.0', 'region_1783.0', 'region_2068.0', 'region_2069.0', 'region_2118.0', 'region_2119.0', 'region_2168.0', 'year_2016.0', 'month_1.0', 'month_2.0', 'month_3.0', 'month_4.0', 'month_5.0', 'month_6.0', 'weekday_0.0', 'weekday_1.0', 'weekday_2.0', 'weekday_3.0', 'weekday_4.0', 'weekday_5.0', 'weekday_6.0', 'hour_0.0', 'hour_1.0', 'hour_2.0', 'hour_3.0', 'hour_4.0', 'hour_5.0', 'hour_6.0', 'hour_7.0', 'hour_8.0', 'hour_9.0', 'hour_10.0', 'hour_11.0', 'hour_12.0', 'hour_13.0', 'hour_14.0', 'hour_15.0', 'hour_16.0', 'hour_17.0', 'hour_18.0', 'hour_19.0', 'hour_20.0', 'hour_21.0', 'hour_22.0', 'hour_23.0']

In [7]:
cols_drop5_num = ['dt_hour']+cols_y+cols_main_cat + cols_polinom #!!! полином здесь не мое изобретение
X_train5_num = data5.drop(['dt_hour']+cols_y+cols_main_cat, axis=1)[:train_endtime] #cols_main_num
X_tune5_num  = data5.drop(['dt_hour']+cols_y+cols_main_cat, axis=1)[tune_begtime:tune_endtime]
X_test5_num  = data5.drop(['dt_hour']+cols_y+cols_main_cat, axis=1)[test_begtime:test_endtime]

cols_drop5_cat = ['dt_hour']+cols_y+cols_main_num + cols_polinom #!!! полином здесь не мое изобретение
X_train5_cat = data5.drop(['dt_hour']+cols_y+cols_main_num, axis=1)[:train_endtime] #cols_main_num
X_tune5_cat  = data5.drop(['dt_hour']+cols_y+cols_main_num, axis=1)[tune_begtime:tune_endtime]
X_test5_cat  = data5.drop(['dt_hour']+cols_y+cols_main_num, axis=1)[test_begtime:test_endtime]

y_train5, y_tune5, y_test5 = [], [], []
for col in cols_y:
    y_train5.append(data5[[col]][:train_endtime])
    y_tune5.append(data5[[col]][tune_begtime:tune_endtime])
    y_test5.append(data5[[col]][test_begtime:test_endtime])

__Ridge Regression  (5-ая неделя, с числовыми значениями)__

In [39]:
lr = linear_model.Ridge(alpha=250000) 
lr.fit(X_train5_num,y_train5[0])
lr_pred = lr.predict(X_tune5_num)
print 'Rigde Numerical', lr5.score(X_tune5_num,y_tune5[0]), metrics.mean_absolute_error(y_tune5[0], lr.predict(X_tune5_num))

Rigde Numerical 0.9367453201674699 31.78393749453423


__Ridge Regression  (5-ая неделя, с категориальными значениями)__

In [40]:
lr_cat = linear_model.Ridge(alpha=250000) 
lr_cat.fit(X_train5_cat,y_train5[0])
lr_cat_pred = lr_cat.predict(X_tune5_cat)
print 'Rigde Categorial', lr_cat.score(X_tune5_cat,
                                       y_tune5[0]), metrics.mean_absolute_error(y_tune5[0], lr_cat.predict(X_tune5_cat))

Rigde Categorial 0.9364733433747702 31.81899723631606


### Ridge Regression  (6 неделя, с доп. признаками 1-й набор (RatecodeID, VendorID, payment_type и др.)

In [8]:
# Оставим числовые признаки от прошлой недели, особой разницы на 5-й неделе не было, а признаков значительно меньше.
cols_drop6 = ['dt_hour']+cols_y+cols_main_cat #!!! полином здесь не мое изобретение
X_train6 = data6.drop(['dt_hour']+cols_y+cols_main_cat, axis=1)[:train_endtime] #cols_main_num
X_tune6  = data6.drop(['dt_hour']+cols_y+cols_main_cat, axis=1)[tune_begtime:tune_endtime]
X_test6  = data6.drop(['dt_hour']+cols_y+cols_main_cat, axis=1)[test_begtime:test_endtime]

y_train6, y_tune6, y_test6 = [], [], []
for col in cols_y:
    y_train6.append(data6[[col]][:train_endtime])
    y_tune6.append(data6[[col]][tune_begtime:tune_endtime])
    y_test6.append(data6[[col]][test_begtime:test_endtime])

In [9]:
lr6 = linear_model.Ridge(alpha=250000) 
lr6.fit(X_train6,y_train6[0])
lr6_pred = lr6.predict(X_tune6)

print 'Ridge (property set 6.1)', lr6.score(X_tune6,y_tune6[0]), metrics.mean_absolute_error(y_tune6[0], lr6.predict(X_tune6))

C:\Users\anna8\Anaconda2\lib\site-packages\sklearn\linear_model\ridge.py:112: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number1.637915e-20
  overwrite_a=True).T


Ridge (property set 6.1) 0.970413275301081 21.7651486542025


__Во-вторых__, чтобы улучшить качество прогнозов в аномальные периоды, вы можете найти информацию о потенциально влияющих на количество поездок событиях, таких, как государственные праздники. Проанализируйте, как именно поведение пассажиров меняется во время этих событий, и создайте признаки, отражающие эти изменения. Как показывает наш опыт, правильный учёт праздничных дней часто позволяет существенно уменьшить среднюю ошибку прогноза.

https://pypi.org/project/holidays/

In [10]:
data6['date'] = pd.to_datetime(data6.dt_hour.dt.date)

In [12]:
us_holidays = holidays.UnitedStates(state='NY', years=2016)  # or holidays.US(), or holidays.CountryHoliday('US')

In [53]:
type(us_holidays)

holidays.UnitedStates

In [13]:
df_hday = pd.DataFrame(us_holidays.items(), columns=['date','hday_name'])
df_hday['date'] = pd.to_datetime(df_hday.date)
df_hday['hday'] = 1


In [46]:
df_hday.shape

(13, 4)

In [47]:
df_hday.drop(df_hday[df_hday.date > data6.date.max()].index, inplace=True)
df_hday.shape

(5, 4)

In [49]:
df_hday['hday_label'] = df_hday.hday_name.map(lambda x:x[:5]+'..'+x[-5:])

In [50]:
print df_hday.hday_label.unique().shape, df_hday.hday_name.unique().shape

(5L,) (5L,)


In [51]:
df_hday.sort_values(by=['hday_name'])

,date,hday_name,hday,hday_label
0,2016-02-12,Lincoln's Birthday,1,Linco..thday
9,2016-01-18,"Martin Luther King, Jr. Day",1,Marti... Day
12,2016-05-30,Memorial Day,1,Memor..l Day
6,2016-01-01,New Year's Day,1,New Y..s Day
2,2016-02-15,"Washington's Birthday, Susan B. Anthony Day",1,Washi..y Day


In [40]:
print data6.date.dtype, df_hday.date.dtype

datetime64[ns] datetime64[ns]


In [52]:
data6['holiday'] = np.array(pd.merge(data6, df_hday, how = 'left', on = ['date']).hday.fillna(0), int)
data6['hday_lbl'] = pd.merge(data6, df_hday, how = 'left', on = ['date']).hday_label.fillna(0), int)

In [60]:
data6['hday_lbl'] = list(pd.merge(data6, df_hday, how = 'left', on = ['date']).hday_label.fillna('General'))

In [62]:
data6.drop('hday_lbl', axis=1, inplace=True)

In [63]:
data6['hday'] = list(pd.merge(data6, df_hday, how = 'left', on = ['date']).hday_label.fillna('UsualDay'))

In [59]:

data6.holiday.value_counts()

0    432684
1     12138
Name: holiday, dtype: int64

In [73]:
cols_hday_cat = ['hday_' + x for x in list(data6.hday.value_counts().index)]

In [75]:
dfm_hday_cat[cols_hday_cat][:2]

,hday_UsualDay,hday_Linco..thday,hday_Marti... Day,hday_Washi..y Day,hday_Memor..l Day,hday_New Y..s Day
2016-01-01 01:00:00,0,0,0,0,0,1
2016-01-01 02:00:00,0,0,0,0,0,1


In [82]:
print cols_hday_cat

['hday_UsualDay', 'hday_Linco..thday', 'hday_Marti... Day', 'hday_Washi..y Day', 'hday_Memor..l Day', 'hday_New Y..s Day']


In [76]:
data6 = pd.get_dummies(data6, columns=['hday'])

In [81]:
data6[cols_hday_cat][:2]

,hday_UsualDay,hday_Linco..thday,hday_Marti... Day,hday_Washi..y Day,hday_Memor..l Day,hday_New Y..s Day
2016-01-01 01:00:00,0,0,0,0,0,1
2016-01-01 02:00:00,0,0,0,0,0,1


In [78]:
data6.holiday.value_counts()

0    432684
1     12138
Name: holiday, dtype: int64

In [79]:
# Оставим числовые признаки от прошлой недели, особой разницы на 5-й неделе не было, а признаков значительно меньше.
cols_drop62 = ['dt_hour', 'date']+cols_y+cols_main_cat #!!! полином здесь не мое изобретение
X_train62 = data6.drop(cols_drop62, axis=1)[:train_endtime] #cols_main_num
X_tune62  = data6.drop(cols_drop62, axis=1)[tune_begtime:tune_endtime]
X_test62  = data6.drop(cols_drop62, axis=1)[test_begtime:test_endtime]

In [80]:
lr62 = linear_model.Ridge(alpha=250000) 
lr62.fit(X_train62,y_train6[0])
lr62_pred = lr62.predict(X_tune62)
print 'Ridge Holidays (6.2)', lr62.score(X_tune62,y_tune6[0]), metrics.mean_absolute_error(y_tune6[0], lr62.predict(X_tune62))

C:\Users\anna8\Anaconda2\lib\site-packages\sklearn\linear_model\ridge.py:112: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number1.588551e-20
  overwrite_a=True).T


Ridge Holidays (6.2) 0.9704145132166917 21.761771117389504


__Увы, улчшения от праздников не получилось__, скорее всего это можно объяснить тем, что у меня всего один год в выборке, и вряд ли этот признак здесь полезен

В-третьих, можно использовать признаки, связанные с географией. Например, скорее всего, суммарное количество поездок, совершаемых из географической зоны, пропорционально площади этой зоны. Для зон, прилегающих к аэропорту, может быть характерен специфический паттерн дневной сезонности, связанный с тем, что спрос на такси будет повышаться в те часы, когда общественный транспорт перестаёт работать. В деловом центре максимальное количество поездок будет приходиться на начало и окончание рабочего дня, на Бродвее — на время начала и окончания спектаклей. Все эти идеи не обязательно верны, мы приводим их здесь только для того, чтобы продемонстрировать принцип рассуждений. Ещё один пример географического признака: можно попробовать добавить идентификатор боро, который можно найти в файле https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv. Кроме того, нам кажется перспективным использование в качестве фактора количества поездок, совершённых за прошлый час/день и т. д. из соседних географических зон, или количества поездок, совершённых за прошлый час/день в текущую географическую зону.

### Поездки соседних районов

In [85]:
#Загружаем районы
dfreg = pd.read_csv('regions.csv', sep=';') 
dfreg[:2]

,region,west,east,south,north
0,1,-74.25559,-74.244478,40.496120,40.504508
1,2,-74.25559,-74.244478,40.504508,40.512896


In [273]:
#данные в виде двумерного массива (строки-y, стобцы-x)
rm = np.array(dfreg.sort_values('region').region).reshape([50,50]) 
rm_line = np.zeros([50,1], int)
#для определения соседних районов сместим матрицу влево-вправо-верх-вниз, тогда на преждней  позиции района окажется сосед
rm_r    = np.hstack([rm[:, 1:], rm_line])     #сосед справа
rm_l    = np.hstack([rm_line, rm[:, :-1]])    #сосед слева
rm_dn   = np.vstack([rm[1:, :], rm_line.T])   #сосед снизу
rm_up   = np.vstack([rm_line.T, rm[:-1, :]])  #сосед сверху 
rm_ldn  = np.hstack([rm_line, rm_dn[:, :-1]]) #сосед слева-снизу
rm_lup  = np.hstack([rm_line, rm_up[:, :-1]]) #сосед слева-сверху
rm_rdn  = np.hstack([rm_dn[:, 1:], rm_line])  #сосед справа-снизу
rm_rup  = np.hstack([rm_up[:, 1:], rm_line])  #сосед справа-сверху

In [341]:
dic_reg_next = {}
for i in range(rm.shape[0]):
    for j in range(rm.shape[1]):
        reg_next0 = [rm_r[i, j] , rm_l[i, j] , rm_dn[i,j], rm_up[i,j], rm_ldn[i,j], rm_lup[i,j], rm_rdn[i,j], rm_rup[i,j]]
        reg_next = [x for x in reg_next0 if x != 0]
        if rm[i, j] in list(actual_regs.region):
            dic_reg_next[rm[i, j]] = reg_next

In [342]:
len(dic_reg_next)

102

In [276]:
data6['near_trips'] = 0
for r in tqdm_notebook(dic_reg_next.items()):
    reg_num  = r[0]
    reg_near = r[1]
    np_near = np.array(data6[data6.region.isin(reg_near)].groupby(['dt_hour'], as_index=False).trips.sum().trips, int)
    data6.loc[data6.region == reg_num, 'near_trips'] = np.array(np_near, int)

In [279]:
data6['prev_date'] = data6.dt_hour - pd.Timedelta(hours=1)

In [289]:
%%time
data6.loc[:, 'prev_near_trips'] = np.array(pd.merge(data6, data6, how='left', left_on=['region', 'prev_date'], 
                                                   right_on=['region','dt_hour'])['near_trips_y'].fillna(0), int)

Wall time: 3.48 s


In [296]:
# Оставим числовые признаки от прошлой недели, особой разницы на 5-й неделе не было, а признаков значительно меньше.
cols_drop63 = ['dt_hour', 'date','prev_date']+cols_y+cols_main_cat #!!! полином здесь не мое изобретение
X_train63 = data6.drop(cols_drop63, axis=1)[:train_endtime] #cols_main_num
X_tune63  = data6.drop(cols_drop63, axis=1)[tune_begtime:tune_endtime]
X_test63  = data6.drop(cols_drop63, axis=1)[test_begtime:test_endtime]

In [297]:
lr63 = linear_model.Ridge(alpha=250000) 
lr63.fit(X_train63,y_train6[0])
lr63_pred = lr63.predict(X_tune63)
print 'Ridge Neartrips (6.3)', lr63.score(X_tune63,y_tune6[0]), metrics.mean_absolute_error(y_tune6[0], lr63.predict(X_tune63))
# 0.970413275301081 21.7651486542025

Ridge Neartrips (6.3) 0.9714671386992391 21.583327071471732


In [440]:
print 'Улучшение (поездки соседних районов):', 21.7651486542025 - 21.583327071471732

Улучшение (поездки соседних районов): 0.181821582731


#### Стало несколько лучше

### Аэропорт

In [307]:
reg_airport = dfreg[(dfreg.west < -73.7781391) & 
                    (dfreg.east > -73.7781391) & (dfreg.south < 40.6413111) & (dfreg.north > 40.6413111)].region.values[0]

In [355]:
print reg_airport

2118


In [400]:
pd.DataFrame(rm).iloc[42:48, 15:20]

,15,16,17,18,19
42,2116,2117,2118,2119,2120
43,2166,2167,2168,2169,2170
44,2216,2217,2218,2219,2220
45,2266,2267,2268,2269,2270
46,2316,2317,2318,2319,2320
47,2366,2367,2368,2369,2370


In [311]:
# Район аэропорта
data6['airport_area'] = 0
data6.loc[data6.region.isin(dic_reg_next[reg_airport]), 'airport_area'] = 1

In [418]:
#Время работы групповых шатлов: 5.30 23:30
data6['airport_shuttle'] = 0
data6.loc[(data6['airport_area']==1) & (data6.hour >= 6) & (data6.hour <= 23), 'airport_shuttle'] = 1

In [424]:
cols_drop64 = ['dt_hour', 'date','prev_date']+cols_y+cols_main_cat
X_train64 = data6.drop(cols_drop64, axis=1)[:train_endtime]
X_tune64  = data6.drop(cols_drop64, axis=1)[tune_begtime:tune_endtime]
X_test64  = data6.drop(cols_drop64, axis=1)[test_begtime:test_endtime]

In [428]:
lr64 = linear_model.Ridge(alpha=250000) 
lr64.fit(X_train64,y_train6[0])
lr64_pred = lr64.predict(X_tune64)
print 'Ridge Neartrips (6.4)', lr64.score(X_tune64,y_tune6[0]), metrics.mean_absolute_error(y_tune6[0], lr64.predict(X_tune64))

Ridge Neartrips (6.4) 0.9714671959181759 21.58316822352148


In [441]:
print 'Улучшение (выделена зона аэропорта):', 21.583327071471732-21.583276409933198

Улучшение (выделена зона аэропорта): 5.06615385341e-05


#### Стало еще немного лучше

### Выбор модели

В прошлом задании я остановился на модели гремневой регрессии. Однако дополнительные признаки, улучшив результат в абсолбте, все-таки не сделали его достаточно хорошим. Попроюуем другие модели.

__Gradient Boosting Regressor__

In [446]:
%%time
bgr = ensemble.GradientBoostingRegressor().fit(X_train6,y_train6[0])

C:\Users\anna8\Anaconda2\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Wall time: 2min 43s


In [498]:
print(metrics.mean_absolute_error(y_tune6[0], bgr.predict(X_tune6)))

20.509643306020326


__Долго обучается, но результат значительно лучше!__

In [502]:
%%time
rf64 = ensemble.RandomForestRegressor() 
rf64.fit(X_train64,y_train6[0])
rf64_pred = rf64.predict(X_tune64)

C:\Users\anna8\Anaconda2\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


Wall time: 2min 18s


In [503]:
print metrics.mean_absolute_error(y_tune6[0], rf64_pred)

16.600332988405103


__Обучается тоже долго, но результат хороший! Оставим эту модель__

In [488]:
lr64_list = []
Q_list = []
for i in tqdm_notebook(range(6)):
    lr64 = linear_model.Ridge(alpha=250000) 
    lr64.fit(X_train64,y_train6[i])
    lr64_pred = lr64.predict(X_tune64)
    lr64_list.append(lr64)
    Q_list.append(metrics.mean_absolute_error(y_tune6[i], lr64_pred))

Q_tune = sum(Q_list) / 6
print 'Ошибка прогноза:', Q_tune

Ошибка прогноза: 33.85872499567409


In [505]:
rf64_list = []
rfQ_list = []
for i in tqdm_notebook(range(6)):
    rf64 = ensemble.RandomForestRegressor() 
    rf64.fit(X_train64.values,y_train6[i].values)
    rf64_pred = rf64.predict(X_tune64)
    rf64_list.append(rf64)
    rfQ_list.append(metrics.mean_absolute_error(y_tune6[i], rf64_pred))

rfQ_tune = sum(rfQ_list) / 6
print 'Ошибка прогноза:', rfQ_tune

C:\Users\anna8\Anaconda2\lib\site-packages\ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """
C:\Users\anna8\Anaconda2\lib\site-packages\ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """
C:\Users\anna8\Anaconda2\lib\site-packages\ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """
C:\Users\anna8\Anaconda2\lib\site-packages\ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """
C:\Users\anna8\Anaconda2\lib\site-packages\ipykernel_launcher.py:5: DataConversionWa

Ошибка прогноза: 20.297729664712076


In [508]:
fin_pred_list = []
fin_kaggle = []
X_regions =  X_test64.region.values 
for i in tqdm_notebook(range(6)):
    fin_pred = rf64_list[i].predict(X_test64)
    fin_pred_list.append(fin_pred)
    for j, t in enumerate(X_test64.index):
        region = X_regions[j]
        idx = '_'.join([str(int(region)),str(t.date()),str(t.hour),str(i+1)])
        y = str((fin_pred[j]))
        answer = ','.join([idx, y])
        fin_kaggle.append(answer)

print('\n'.join(predict_kaggle[:10]))

1075_2016-05-31_23_1,[11.80525603]
1075_2016-06-01_0_1,[14.2365185]
1075_2016-06-01_1_1,[0.64724212]
1075_2016-06-01_2_1,[-3.92622262]
1075_2016-06-01_3_1,[1.84472852]
1075_2016-06-01_4_1,[4.47648728]
1075_2016-06-01_5_1,[14.49545111]
1075_2016-06-01_6_1,[43.9358622]
1075_2016-06-01_7_1,[53.98920525]
1075_2016-06-01_8_1,[63.52353031]


In [515]:
Qs = []
for i in tqdm_notebook(range(6)):
    fin_pred = fin_pred_list[i] 
    Qs.append(metrics.mean_absolute_error(y_test6[i], fin_pred))

print sum(Qs) / 6

19.929226884226885


In [517]:
with open(data6_path + 'Alexey_Bebchik_k6w6_answer3.csv','w') as f:
    f.write('id,y\n')
    f.write('\n'.join(fin_kaggle))

### ВЫВОД

В работе добавлены:
1 дополнительные признаки из сырых данных (тариф,вендор и др.);  
2 внешние признаки праздников;  
3 географические признаки (данные соседних районов, район аэропорта).  

Качество прогноза значительно выросло после добавления первой группы признаков,  еще несколько выросло третьей группе. Праздники должного эффекта не дали. Качество обучение проверялось на гребневой регрессии, подобранной в прошлом задании. 

Далее было попробованы еще две модели - градиентный бустинг и случайный лес. Случайный лес справился лучше, на нем и выполнен финальный расчет проноза.

Файл ответа загружен на Kaggle: AlexModern, 63 место (score __19.92921__) Приложен скриншот:   

__Alexey_Bebchik_Kaggle_Leaderbord_Week6(AlexModern)__

In [467]:
Q_list_test = []
for i in tqdm_notebook(range(6)):
    lr64_pred_test = lr64_list[i].predict(X_test64)
    Q_list_test.append(metrics.mean_absolute_error(y_test6[i], lr64_list[i].predict(X_test64)))

Q_test = sum(Q_list_test) / 6
print 'Ошибка прогноза:', Q_test

Ошибка прогноза: 45.111395752276906


In [ ]:
# формируем идентификатор: Регион_ДатаЧас_ЧасПрозноза
pred['save_id'] = pred.region.astype(int).astype(str) + '_'+pred.dt_hour.dt.strftime('%Y-%m-%d_%H')
# если в часе даты конца истории встречается 0, например, 03, то надо оставить 3 (если 00, то будет 0)
pred['save_id'] = pred.save_id.apply(lambda x:x[:-2]+x[-1:] if x[-2:-1]=='0' else x)
# добавляем номер прогнозного часа от 1 до 6
pred['save_id'] = pred['save_id'] + '_' + pred.num6.astype(str) #.apply(lambda x:x[-1:])

### Дополнительные признаки

In [167]:
regs = data6.region.unique()

In [170]:
regs[:5]

array([ 1075.,  1076.,  1077.,  1125.,  1126.])

trip_len, passenger_count, trip_distance, region2, RatecodeID, payment_type, fare_amount, VendorID


trip_len = tpep_dropoff_datetime - tpep_pickup_datetime средняя длительность поездок;      
passenger_count среднее количество пассажиров;      
trip_distance среднее расстояние по счётчику;      
region2 = dropoff_longitude+dropoff_latitude доли географических зон, в которые совершаются поездки;      
RatecodeID доли поездок, совершаемых по тарифам каждого из типов;      
payment_type доли способов оплаты поездок;      
fare_amount средняя стоимость поездок;      
VendorID доли провайдеров данных.    

ndex([u'VendorID', u'tpep_pickup_datetime', u'tpep_dropoff_datetime',  
       u'passenger_count', u'trip_distance', u'pickup_longitude',
       u'pickup_latitude', u'RatecodeID', u'store_and_fwd_flag',
       u'dropoff_longitude', u'dropoff_latitude', u'payment_type',
       u'fare_amount', u'extra', u'mta_tax', u'tip_amount', u'tolls_amount',
       u'improvement_surcharge', u'total_amount'],
      dtype='object')

In [69]:
actual_regs = pd.read_csv(data6_path + 'YTw6_reg.csv')

In [5]:
#Загрузим данные, ограничив их по району, количеству пассажиров и продолжительности поездки
data_cols = ['tpep_pickup_datetime','tpep_dropoff_datetime','passenger_count', 'trip_distance','pickup_longitude',
             'pickup_latitude'] + ['dropoff_longitude', 'dropoff_latitude', 'RatecodeID' , 'payment_type', 
                                   'fare_amount', 'total_amount', 'VendorID']
cols_cut = ['dt_hour', 'region', 'region2', 'passenger_count', 'trip_distance', 
             'RatecodeID', 'payment_type', 'fare_amount', 'total_amount', 'VendorID']

In [21]:
%%time 
save_fname = 'k6w6_data_ext.csv', 
month_count = 6
reg_fname='regions.csv'
cut_regLess5 = True
save_data=False #True
# return_data=True

#Координаты квадрата Нью_Йорка
west_ny, east_ny, south_ny, north_ny  = -74.25559,-73.700018, 40.49612, 40.91553

#Загружаем районы и получаем границы корзин
regs = pd.read_csv(data_path + 'regions.csv', sep=';') 
binx  = sorted(set(regs.west.value_counts().index)  | set(regs.east.value_counts().index ))
biny  = sorted(set(regs.south.value_counts().index) | set(regs.north.value_counts().index))

# Составим список файлов для обработки
files = [data_path + 'yellow_tripdata_2016-' + ('0'+str(x+1))[-2:] + '.csv' for x in range(month_count)] [:] #для отладкаи 5

dfmon_list = []
for i, f in enumerate(files): #tqdm_notebook(
#     sleep(0.1)
    print_time('read '+ f)
    
    dfmon = pd.read_csv(f, sep=',', parse_dates=['tpep_pickup_datetime','tpep_dropoff_datetime'], usecols=data_cols) 
    
    #print_time('get del_index')
    
    print_time('create del index')
    del_index = dfmon[(dfmon.tpep_pickup_datetime == dfmon.tpep_dropoff_datetime) #Нулевая длительность
                | (dfmon['passenger_count' ] == 0)          # поездки с нулевым количеством пассажиров
                | (dfmon['trip_distance'   ] == 0)          # поездки с нулевым расстоянием поездки по счётчику
                | (dfmon['pickup_longitude'] <  west_ny)    # поездки, не попадающие в прямоугольник Нью-Йорка
                | (dfmon['pickup_longitude'] >= east_ny)    
                | (dfmon['pickup_latitude' ] <  south_ny)   
                | (dfmon['pickup_latitude' ] >= north_ny)  
                     ].index
    #print_time('drop del_index')
    dfmon.drop(del_index, inplace=True)
    dfmon['dt_hour'] = dfmon.tpep_pickup_datetime.dt.floor('H') #день и час (.dt.hour - только час)
    #длительность поездки
    dfmon.loc[:,'trip_len'] = ((dfmon.tpep_dropoff_datetime - dfmon.tpep_pickup_datetime) / np.timedelta64(1, 'm')).astype(int)
    
    print_time('binned_statistic_2d ')
    
    #Разобъем данные на квадраты
    r_bins = binned_statistic_2d(
        x = dfmon.pickup_longitude, 
        y = dfmon.pickup_latitude, 
        values = None, 
        statistic = 'count', 
        bins      = [binx, biny],
        range     = [[-74.25559, -73.70001], [40.49612, 40.91553]], expand_binnumbers = True)
    # Вычислим номер региона
    bin_regs = np.array([(x-1) * 50 + y for x, y in zip(*r_bins.binnumber)])
    #print_time('set regs')
    dfmon['region']  = bin_regs

    #Разобъем данные на квадраты
    print_time('binned_statistic_2d 2')
    r_bins2 = binned_statistic_2d(
        x = dfmon.dropoff_longitude, 
        y = dfmon.dropoff_latitude, 
        values = None, 
        statistic = 'count', 
        bins      = [binx, biny],
        range     = [[-74.25559, -73.70001], [40.49612, 40.91553]], expand_binnumbers = True)
    # Вычислим номер региона
    bin_regs2 = np.array([(x-1) * 50 + y for x, y in zip(*r_bins2.binnumber)])
    #print_time('set regs')
    dfmon['region2']  = bin_regs2
    
    print 'drop regs less 5'
    dfmon = dfmon[dfmon.region.isin(list(actual_regs.region))]
    
    print 'saveing file'
    dfmon[cols_cut].to_csv('cut2_'+ f, index=False, chunksize=100000)
    

read yellow_tripdata_2016-01.csv 16:33
create del index 16:34
binned_statistic_2d  16:35
binned_statistic_2d 2 16:35
drop regs less 5
save file
read yellow_tripdata_2016-02.csv 16:37
create del index 16:39
binned_statistic_2d  16:39
binned_statistic_2d 2 16:39
drop regs less 5
save file
read yellow_tripdata_2016-03.csv 16:41
create del index 16:42
binned_statistic_2d  16:42
binned_statistic_2d 2 16:43
drop regs less 5
save file
read yellow_tripdata_2016-04.csv 16:44
create del index 16:46
binned_statistic_2d  16:46
binned_statistic_2d 2 16:46
drop regs less 5
save file
read yellow_tripdata_2016-05.csv 16:48
create del index 16:49
binned_statistic_2d  16:49
binned_statistic_2d 2 16:50
drop regs less 5
save file
read yellow_tripdata_2016-06.csv 16:51
create del index 16:52
binned_statistic_2d  16:52
binned_statistic_2d 2 16:53
drop regs less 5
save file
Wall time: 21min 8s


In [4]:
month_count = 6
cut_files = [data6_path + 'cut2_yellow_tripdata_2016-' + ('0'+str(x+1))[-2:] + '.csv' for x in range(month_count)]

In [6]:
%%time 
dfmon_list = []
for i, f in enumerate(cut_files): #tqdm_notebook(
    print_time('load '+ f)
    dfmon = pd.read_csv(f, sep=',', parse_dates=['dt_hour']) #, usecols=cols_cut) 
    dfmon_list.append(dfmon)

load cut2_yellow_tripdata_2016-01.csv 09:10
load cut2_yellow_tripdata_2016-02.csv 09:11
load cut2_yellow_tripdata_2016-03.csv 09:11
load cut2_yellow_tripdata_2016-04.csv 09:11
load cut2_yellow_tripdata_2016-05.csv 09:12
load cut2_yellow_tripdata_2016-06.csv 09:12
Wall time: 1min 58s


In [166]:
dfmon_list[0][:2]

,dt_hour,region,region2,passenger_count,trip_distance,RatecodeID,payment_type,fare_amount,total_amount,VendorID
0,2016-01-01,1230,1532,2,5.52,1,2,19.0,20.3,2
1,2016-01-01,1177,1336,2,7.45,1,2,26.0,27.3,2


In [168]:
%%time 
cols_ext_cat  = ['VendorID', 'payment_type', 'RatecodeID']
dfm_agr_list  = []
cols_agr_list = []
for i, dfm in enumerate(dfmon_list):

    # Категориальные признаки 
    cols_main   = list(dfm.columns)
    #     sleep(0.1)
    print_time('file '+ str(i) + ', get_dummies')
    dfm_ext_cat = pd.get_dummies(dfm, columns=cols_ext_cat)

    cols_ext  = list(dfm_ext_cat.columns)[len(cols_main)-len(cols_ext_cat):]
    print_time('file '+ str(i) + ', groupby')
    dfm_agr = dfm_ext_cat.groupby(['dt_hour', 'region'], as_index=False).mean()

    dfm_trips = dfm.groupby(['dt_hour', 'region'], as_index=False).passenger_count.count()

    dfm_agr.loc[:, 'trips'] = dfm_trips.passenger_count
    dfm_agr.loc[:, 'trips'] = np.array(dfm_agr.trips, int)

    cols_agr_list.append(cols_ext1)
    dfm_agr_list.append(dfm_agr)

file 0, get_summies 11:28
file 0, groupby 11:28
file 1, get_summies 11:29
file 1, groupby 11:29
file 2, get_summies 11:29
file 2, groupby 11:30
file 3, get_summies 11:30
file 3, groupby 11:30
file 4, get_summies 11:30
file 4, groupby 11:31
file 5, get_summies 11:31
file 5, groupby 11:32
Wall time: 3min 25s


In [140]:
dfm_trips[:1]

,dt_hour,region,passenger_count
0,2016-06-01,1075,26


In [139]:
dfm_agr[:1]

,dt_hour,region,region2,passenger_count,trip_distance,fare_amount,total_amount,VendorID_1,VendorID_2,payment_type_1,...,payment_type_4,payment_type_5,RatecodeID_1,RatecodeID_2,RatecodeID_3,RatecodeID_4,RatecodeID_5,RatecodeID_6,RatecodeID_99,trips
0,2016-01-01,1075,1203.9,2.0375,3.972625,14.88125,17.590625,0.4375,0.5625,0.425,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,26


In [ ]:
dfm_trips, dfm_agr[:]

In [75]:
dfm_agr_list[0][:2]

,dt_hour,region,region2,passenger_count,trip_distance,fare_amount,total_amount,VendorID_1,VendorID_2,payment_type_1,...,payment_type_3,payment_type_4,payment_type_5,RatecodeID_1,RatecodeID_2,RatecodeID_3,RatecodeID_4,RatecodeID_5,RatecodeID_6,RatecodeID_99
0,2016-01-01,1075,967,1.0,10.00,29.5,32.80,1.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2016-01-01,1075,1076,1.2,0.57,6.0,8.31,0.6,0.4,0.6,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [191]:
%%time 
for i, dfm_agr in enumerate(dfm_agr_list):
    print_time('k6w4_data_ext_' + str(i))
    dfm_agr.to_csv('k6w4_data_ext_' + str(i), index=False)

In [72]:
#все регионы
regions  = pd.DataFrame(actual_regs.region, columns=['region'])
regions['key'] = 0 #для cross join в merge 

In [189]:
%%time 
data_hr_list = []

for i, dfm_agr in enumerate(dfm_agr_list):
    
    print_time('hr' + str(i) +  ' ' +  str(dfm_agr.shape))
    
    #все дни и часы за весь период в данных
    start_date  = dfm_agr.dt_hour.min() #дата начала данных
    end_date    = dfm_agr.dt_hour.max() #дата окончания данных
    dt_hours = pd.DataFrame(pd.date_range(start_date, end_date, freq='H').values, columns=['dt_hour'])
    dt_hours['key'] = 0 #для cross join в merge 

    #все дни и часы для каждого региона (cross join)
    data_hr = pd.merge(regions, dt_hours, how='outer')[['dt_hour','region']]

    print_time('update hr' + str(i))
    
    #добавим столбец с количеством поездок для каждой пары с помощью left join, заполнив NaN нулями
    for c in list(dfm_agr.columns[3:]):
        data_hr[c] = data_hr.merge(dfm_agr, how='left', on = ['dt_hour', 'region']).fillna(0)[c]
    
    print_time('save hr' + str(i) + str(data_hr.shape))
    
    data_hr_list.append(data_hr)
    
    dfm_agr.to_csv(data4_path + 'YTw4_data_ext_hr' + str(i), index=False)   

In [185]:
pd.set_option('display.max_columns', 100)

In [186]:
data_hr_all[:1]

,RatecodeID_1,RatecodeID_2,RatecodeID_3,RatecodeID_4,RatecodeID_5,RatecodeID_6,RatecodeID_99,VendorID_1,VendorID_2,dt_hour,fare_amount,passenger_count,payment_type_1,payment_type_2,payment_type_3,payment_type_4,payment_type_5,region,total_amount,trip_distance,trips
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.4375,0.5625,2016-01-01,14.88125,2.0375,0.425,0.575,0.0,0.0,0.0,1075,17.590625,3.972625,80.0


In [182]:
data_hr_all = pd.concat(data_hr_list)

In [183]:
data_hr_all.shape, data0.shape

((445536, 21), (445536, 3))

In [188]:
data_hr_all.to_csv('k6w4_data_ext_hr.csv', index=False)

__Блок определения пользовательских функций__

In [ ]:
#Готовим данные и записываем их в файл (примерно 6 минут да 2016 год)
def load_prepare_save_data(save_fname, month_count, reg_fname= data_path + 'regions.csv', 
                           cut_regLess5 = True, save_data=True, return_data=True):
    
    #Координаты квадрата Нью_Йорка
    west_ny, east_ny, south_ny, north_ny  = -74.25559,-73.700018, 40.49612, 40.91553
    
    #Загружаем районы и получаем границы корзин
    regs = pd.read_csv(reg_fname, sep=';') 
    binx  = sorted(set(regs.west.value_counts().index)  | set(regs.east.value_counts().index ))
    biny  = sorted(set(regs.south.value_counts().index) | set(regs.north.value_counts().index))
    
    
    # Составим список файлов для обработки
    files = [data_path + 'yellow_tripdata_2016-' + ('0'+str(x+1))[-2:] + '.csv' for x in range(month_count)]# [:1] для отладкаи 5

    #Загрузим данные, ограничив их по району, количеству пассажиров и продолжительности поездки
    data_cols = ['tpep_pickup_datetime','tpep_dropoff_datetime','passenger_count',
                 'trip_distance','pickup_longitude','pickup_latitude'] 

    data_cols = ['tpep_pickup_datetime','tpep_dropoff_datetime','passenger_count', 'trip_distance','pickup_longitude',
                 'pickup_latitude'] + ['dropoff_longitude', 'dropoff_latitude', 'RatecodeID' , 'payment_type', 
                                       'fare_amount', 'total_amount', 'VendorID']
    dfmon_list = []
    for f in files:
        print_time('read '+ f)
        dfmon = pd.read_csv(f, sep=',', parse_dates=['tpep_pickup_datetime','tpep_dropoff_datetime'], usecols=data_cols) 
        #print_time('get del_index')
        del_index = dfmon[(dfmon.tpep_pickup_datetime == dfmon.tpep_dropoff_datetime) #Нулевая длительность
                    | (dfmon['passenger_count' ] == 0)          # поездки с нулевым количеством пассажиров
                    | (dfmon['trip_distance'   ] == 0)          # поездки с нулевым расстоянием поездки по счётчику
                    | (dfmon['pickup_longitude'] <  west_ny)    # поездки, не попадающие в прямоугольник Нью-Йорка
                    | (dfmon['pickup_longitude'] >= east_ny)    
                    | (dfmon['pickup_latitude' ] <  south_ny)   
                    | (dfmon['pickup_latitude' ] >= north_ny)  
                         ].index
        #print_time('drop del_index')
        dfmon.drop(del_index, inplace=True)
        dfmon['dt_hour'] = dfmon.tpep_pickup_datetime.dt.floor('H') #день и час (.dt.hour - только час)
        #print_time('calc reg bins')
        #Разобъем данные на квадраты
        r_bins = binned_statistic_2d(
            x = dfmon.pickup_longitude, 
            y = dfmon.pickup_latitude, 
            values = None, 
            statistic = 'count', 
            bins      = [binx, biny],
            range     = [[-74.25559, -73.70001], [40.49612, 40.91553]], expand_binnumbers = True)
        # Вычислим номер региона
        bin_regs = np.array([(x-1) * 50 + y for x, y in zip(*r_bins.binnumber)])
        #print_time('set regs')
        dfmon['region']  = bin_regs
        #print_time('aggregate')
        dfmon_agr = dfmon.groupby(['dt_hour', 'region'], as_index=False).pickup_longitude.count()
        dfmon_agr.columns = ['dt_hour', 'region', 'trips']
        dfmon_list.append(dfmon_agr)

    data_trips = pd.concat(dfmon_list) #только совершенные поездки в регионах

    #---- надо добавить нули для регионов и часов, в которых не было поездок

    #все регионы
    regions  = pd.DataFrame(regs.region, columns=['region'])
    regions['key'] = 0 #для cross join в merge 

    #все дни и часы за весь период в данных
    start_date  = data_trips.dt_hour.min() #дата начала данных
    end_date    = data_trips.dt_hour.max() #дата окончания данных
    dt_hours = pd.DataFrame(pd.date_range(start_date, end_date, freq='H').values, columns=['dt_hour'])
    dt_hours['key'] = 0 #для cross join в merge 

    #все дни и часы для каждого региона (cross join)
    data_hr = pd.merge(regions, dt_hours, how='outer')[['dt_hour','region']]

    #добавим столбец с количеством поездок для каждой пары с помощью left join, заполнив NaN нулями
    data_hr['trips'] = data_hr.merge(data_trips, how='left', on = ['dt_hour', 'region']).fillna(0).trips.astype(int)

    #Подсчитываем среднее число поездок в час (по данным за май 2016, согласно 2-му заданию, оставляем 102 района)
    if cut_regLess5 == True:
        data_hr_mean = data_hr[(data_hr.dt_hour.dt.year==2016)&
                               (data_hr.dt_hour.dt.month==5)].groupby('region', as_index=False)['trips'].mean()
        regs_less5 = data_hr_mean[data_hr_mean.trips<5].region.values           #районы, где среднее число поездок меньше 5
        data_hr.drop(data_hr[data_hr.isin({'region':regs_less5}).region ].index, inplace=True) #удаляем данные этих районов

    #сохраняем файл
    if save_data == True and save_fname <> '':
        data_hr.to_csv(save_fname, index=False)
    
    #в принципе не обязательно, при выходе из функции сборка мусора удалит переменные
#     del data_hr, dt_hours, data_trips, dfmon_list, data_hr_mean

    if return_data == True:
        return data_hr
